### **Definir el directorio de trabajo con Google Colab**
Define el directorio de trabajo como la carpeta `data` de la carpeta compartida `DS4A-Team12` de Drive.

In [ ]:
import os
from google.colab import drive 
# Enlazar a la carpeta working_directory de la carpeta compartida en Drive
drive.mount('/content/drive')
working_dir = '/content/drive/MyDrive/DS4A-Team12/data'
os.chdir(working_dir)
!pwd

Mounted at /content/drive
/content/drive/MyDrive/DS4A-Team12/data


In [ ]:
%%capture
!pip install jupyter-dash
!pip install dash
!pip install dash-renderer
!pip install dash_html_components
!pip install dash_core_components
!pip install dash_bootstrap_components
!pip install duckdb
!pip install plotly

In [ ]:
# Librerias relevantes 
import dash
import plotly.express as px
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import joblib

### **Abrir base de datos de tomas nutricionales del ICBF (Dirección de Primera Infancia)**
Abre las primeras 98 columnas de todos los registros de la base de datos, preprocesados para asignarle a cada variable el tipo de dato correspondiente (archivo `TomasDPIpre.parquet` de 472.4 MB) como un dataframe de nombre `tom`.
Para preprocesar un archivo ir a [`Preprocesamiento_Tomas_.ipynb`](https://https://colab.research.google.com/drive/1ztyI5yfqs6LY7LOYXHr0q4WCrDdfotBs#scrollTo=KJM1uKZq71HA).

In [ ]:
tom = pd.read_parquet('preprocessed_data/tomas.parquet')
points = pd.read_csv('auxiliary_data/puntos_peso_talla.csv')

In [ ]:
app = JupyterDash(__name__)
#app = dash.Dash()

In [ ]:
IdB = 10058
tom[tom['IdBeneficiario'] == IdB]

,IdToma,Registro,Vigencia,Toma,Servicio,FechaValoracionNutricional,EdadMeses,FechaMedicionPerimetroBraquial,MedicionPerimetroBraquial,Peso,Talla,ZScoreTallaEdad,ZScorePesoEdad,ZScorePesoTalla,ZScoreIMC,EstadoTallaEdad,EstadoPesoEdad,EstadoPesoTalla,EstadoIMC,Flag,FechaRegistroSaludNutricion,PresentaCarneVacunacion,ControlesCrecimDesarrollo,AntecedentePremadurez,Direccion,IdBeneficiario,Id
13,13,988320,2017,3,SERVICIO ESPECIAL PARA LA PRIMERA INFANCIA - G...,2017-07-12,64.0,NaT,NaN,24.299999,121.400002,2.10,NaN,NaN,0.77,Talla adecuada para la edad,NaN,NaN,Adecuado para la edad,0,NaT,S,0,NaN,Primera Infancia,10058,<NA>
14,14,988320,2017,4,SERVICIO ESPECIAL PARA LA PRIMERA INFANCIA - G...,2017-10-20,67.0,NaT,NaN,23.400000,123.300003,2.14,NaN,NaN,0.09,Talla adecuada para la edad,NaN,NaN,Adecuado para la edad,0,NaT,S,0,NaN,Primera Infancia,10058,<NA>


In [ ]:
tom.IdBeneficiario[10:100].unique()

<IntegerArray>
[  8872,  10058,  15997,  33714,  33748,  39848,  50556,  51712,  54479,
  60052,  68400,  73858,  82368,  82480,  90194, 111347, 113231, 118954,
 143498, 167610]
Length: 20, dtype: UInt32

In [ ]:
zscores = [i - 3 for i in range(7)]
color_palette = ['green', 'goldenrod', 'red', 'black']
color_estado = {tom.EstadoPesoTalla.cat.categories[i]: color_palette[abs(i - 3)] 
                for i in range(7)}
cols_estado = [col for col in tom.columns if col[:6] == 'Estado']


In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
IdB = 12600290
#IdB = 10058
df = tom[tom['IdBeneficiario'] == IdB][['EdadMeses', 'FechaValoracionNutricional', 
                                        'Talla', 'Peso', 'ZScorePesoTalla'] +
                                       cols_estado].dropna()
if len(df) < 1:
  print('Error: Beneficiario sin muestras útiles')
#df_plot = df[['EdadMeses', 'FechaValoracionNutricional',
 #             'Talla', 'Peso', 'ZScorePesoTalla', 'EstadoPesoTalla']].dropna()
sex = 'M'
ultimo = len(df) - 1

fig = px.scatter(df, x='Talla', y='Peso', color='EstadoPesoTalla',
                 hover_data=['EstadoPesoTalla', 'ZScorePesoTalla', 'EdadMeses'],
                 hover_name='FechaValoracionNutricional',
                 color_discrete_map=color_estado)
fig.add_trace(go.Scatter(x=df['Talla'], y=df['Peso'],
                         mode='lines', showlegend=False,
                         line=dict(color='black', width=1)))
fig.update_xaxes(range=[45, 120])
fig.update_yaxes(range=[0, 34])
for z in zscores:
  fig.add_trace(go.Scatter(x=points['height'], y=points[f'weight_{sex}_{z}'],
                           hoverinfo = 'none', showlegend=False,
                           line=dict(color=color_palette[abs(z)], width=1)))

# Set templates
fig.update_layout(template="plotly_white")
 
app.layout = html.Div(children=[
    html.H1(children='Evolución estado nutricional'),             
             
    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])
app.run_server(mode='inline')

/usr/local/lib/python3.7/dist-packages/plotly/express/_core.py:286: FutureWarning:

Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.



<IPython.core.display.Javascript object>

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
IdB = 12600290
df = tom[tom['IdBeneficiario'] == IdB]
sex = 'M'
ultimo = len(df) - 1

riesgo3ficticio = df['EdadMeses'].iloc[ultimo] * (6 - df['ZScorePesoTalla'].iloc[ultimo]) / 4
riesgo6ficticio = df['EdadMeses'].iloc[ultimo] * (6 - df['ZScoreIMC'].iloc[ultimo]) / 4

fig = px.scatter(df, x='Talla', y="Peso", color="EstadoPesoTalla",
                 hover_data=['EstadoPesoTalla', 'ZScorePesoTalla', 'EdadMeses'],
                 hover_name='FechaValoracionNutricional',
                 color_discrete_map=color_estado)
fig.update_xaxes(range=[45, 120])
fig.update_yaxes(range=[0, 34])
zscores = [i - 3 for i in range(7)]
for z in zscores:
  fig.add_trace(go.Scatter(x=points['height'], y=points[f'weight_{sex}_{z}'],
                           name = '',
                           line=dict(color=color_palette[abs(z)], width=1)))

# Set templates
fig.update_layout(template="plotly_white")
 
app.layout = html.Div(children=[
    html.H1(children='Evolución estado nutricional'),
 
    html.Div(children=f'''
        Identificador: {IdB}, Fecha de Nacimiento: ##.##.##, Sexo: ##, Edad: ## meses,
    '''),
 
    html.H3(children='Última revisión'),
    html.Div(children=f"Fecha: {df['FechaValoracionNutricional'].iloc[ultimo]}"),
    html.Div(children=f"Edad: {df['EdadMeses'].iloc[ultimo]} meses"),
    html.Div(children=f"Talla: {df['Talla'].iloc[ultimo]:.1f} cm"),
    html.Div(children=f"Peso: {df['Peso'].iloc[ultimo]:.1f} kg"),
    html.Div(children=f"Estado Peso/Talla: {df['EstadoPesoTalla'].iloc[ultimo]}"),
    html.Div(children=f"Estado Peso/Edad: {df['EstadoPesoEdad'].iloc[ultimo]}"),
    html.Div(children=f"Estado IMC: {df['EstadoIMC'].iloc[ultimo]}"),
    html.Div(children=f"Estado Talla/Edad: {df['EstadoTallaEdad'].iloc[ultimo]}"),
             
             
    dcc.Graph(
        id='example-graph',
        figure=fig
    ),
    html.H3(children='Riesgo de caer en desnutrición aguda:'),
    html.Div(children=f"En los próximos 3 meses: {round(riesgo3ficticio)}%"),
    html.Div(children=f"En los próximos 6 meses: {round(riesgo6ficticio)}%"),
])
app.run_server(mode='inline')

/usr/local/lib/python3.7/dist-packages/plotly/express/_core.py:286: FutureWarning:

Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.



<IPython.core.display.Javascript object>

In [ ]:

import dash
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
from dash.dependencies import Input, Output
import os
import pandas as pd
import numpy as np
import duckdb
import time

#from app import app
import dash
import dash_bootstrap_components as dbc
app = JupyterDash(__name__)
#app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
app.config.suppress_callback_exceptions = True 
app.title = 'DS4A - Team 12'
server = app.server



inicio = time.time()
dt = pd.read_parquet('preprocessed_data/TomasDPImelted.parquet') #, engine='fastparquet')
fin = time.time()
print(f"Tiempo de ejecucion: {fin-inicio}")
dc = dt.sample(3000)
df = dc.select_dtypes(include= np.number)
available_indicators = df.columns

layout = html.Div([
    html.H4("Scatterplots - Dataset Variables", className="display-4"),
	dbc.Card([
	   #html.Div([
		dbc.CardHeader([
			html.Div([
				dcc.Dropdown(
					id='xaxis-column',
					options=[{'label': i, 'value': i} for i in available_indicators],
					value='Peso'
				),
				dbc.RadioItems(
					className = "form-check-inline pr-10",
					id='xaxis-type',
					options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
					value='Linear',
					labelStyle={'display': 'inline-block'}
				)
			],style={'width': '48%', 'display': 'inline-block'}),

			html.Div([
				dcc.Dropdown(
					id='yaxis-column',
					options=[{'label': i, 'value': i} for i in available_indicators],
					value='Talla'
				),
				dbc.RadioItems(
					className = "form-check-inline",
					id='yaxis-type',
					options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
					value='Linear',
					labelStyle={'display': 'inline-block, padding-right: 10px;'}
				)
			],style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
		]),
		dbc.CardBody(
			dcc.Graph(id='indicator-graphic', style={'width': '100%', 'display': 'inline-block'})
		)
	])
])


@app.callback(
    Output('indicator-graphic', 'figure'),
    [Input('xaxis-column', 'value'),
     Input('yaxis-column', 'value'),
     Input('xaxis-type', 'value'),
     Input('yaxis-type', 'value')])
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type):
    dff = df

    return {
        'data': [dict(
            x = dff[xaxis_column_name],
			y = dff[yaxis_column_name],
			text= dff['IdBeneficiario'],
            mode='markers',
			marker_color=dff['Talla'],
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'white'}
            }
        )],
        'layout': dict(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 40, 't': 10, 'r': 0},
            hovermode='closest'
        )
    }
app.run_server(mode='inline')

In [ ]:
text_input = html.Div(
    [
        html.Div("Digite el NIUP del beneficiario"),
        dbc.Input(id="input", placeholder="NIUP del beneficiario", type="text"),
        html.Br(),
        html.P(id="output"),
    ]
)

app.layout = html.Div([
  html.H1("Hello World"),
  html.Div(f"Evolución de cada individuo {IdB}"),
  text_input
])

@app.callback(Output("output", "children"), [Input("input", "IdB")])
def output_text(IdB):
    return IdB

app.run_server(mode='inline')

#if __name__ == '__main__':
    #app.run_server(debug=True)

In [ ]:
!pip install app

In [ ]:

import dash
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
from dash.dependencies import Input, Output
import os
import pandas as pd
import numpy as np
import duckdb
import time

#import app

inicio = time.time()
dt = tom
fin = time.time()
print(f"Tiempo de ejecucion: {fin-inicio}")
dc = dt.sample(3000)
df = dc.select_dtypes(include= np.number)
available_indicators = df.columns

app.layout = html.Div([
    html.H4("Scatterplots - Dataset Variables", className="display-4"),
	dbc.Card([
	   #html.Div([
		dbc.CardHeader([
			html.Div([
				dcc.Dropdown(
					id='xaxis-column',
					options=[{'label': i, 'value': i} for i in available_indicators],
					value='Peso'
				),
				dbc.RadioItems(
					className = "form-check-inline pr-10",
					id='xaxis-type',
					options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
					value='Linear',
					labelStyle={'display': 'inline-block'}
				)
			],style={'width': '48%', 'display': 'inline-block'}),

			html.Div([
				dcc.Dropdown(
					id='yaxis-column',
					options=[{'label': i, 'value': i} for i in available_indicators],
					value='Talla'
				),
				dbc.RadioItems(
					className = "form-check-inline",
					id='yaxis-type',
					options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
					value='Linear',
					labelStyle={'display': 'inline-block, padding-right: 10px;'}
				)
			],style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
		]),
		dbc.CardBody(
			dcc.Graph(id='indicator-graphic', style={'width': '100%', 'display': 'inline-block'})
		)
	])
])


@app.callback(
    Output('indicator-graphic', 'figure'),
    [Input('xaxis-column', 'value'),
     Input('yaxis-column', 'value'),
     Input('xaxis-type', 'value'),
     Input('yaxis-type', 'value')])
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type):
    dt= df

    return {
        'data': [dict(
            x = dt[xaxis_column_name],#x = dff[xaxis_column_name],
			y = dt[yaxis_column_name],#y = dff[yaxis_column_name],
			text= dt['IdBeneficiario'],#text= dff['IdBeneficiario'],
            mode='markers',
      marker_color=dt['Talla'],#marker_color=dff['Talla'],
            marker={
                'size': 15,
                'opacity': 0.5,
                'line': {'width': 0.5, 'color': 'white'}
            }
        )],
        'layout': dict(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 40, 't': 10, 'r': 0},
            hovermode='closest'
        )
    }


In [ ]:
app.run_server(mode='inline')

In [ ]:
if __name__ == '__main__':
  app.run_server(port = 4050)